In [1]:
from game2 import EKGameV0, Cards
import random

def randomAction(gameState):
    allowedActions = gameState.allowedActions
    return allowedActions[random.randint(0, len(allowedActions)-1)]

def test1(test_fun):
    debug = False
    game = EKGameV0(debug=debug)
    state = game.reset()
    done = False
    # clear_output(wait=True)
    while not done:
        input_action = randomAction(game.gameState)
        if debug:
            print("Observation:", state)
            print("State:", game.gameState.stringifyFullStateExceptDeck())
            print('Action selected:', input_action, Cards(input_action))
        (state, reward, done, info) = game.step(input_action)
        if debug:
            print("reward:", reward)
    # test_fun(state, reward)

def check1(state, reward):
    # print(state, reward)
    # ekstate.printAll()
    if reward == -1:
        # if lost game, ensure that there are no difuses in agent hand
        assert state.currHand[0] == 0
    elif reward == 1:
        # if won game, ensure that there are no difuses in own hand
        assert state.oppHand[0] == 0
    else:
        # shouldnt have any other reward
        assert 1 == 2

num_ep = 100000
for i in range(num_ep):
    if i % 100 == 0:
        print("Episode:", i)
    test1(check1)
print("PASSED")

Episode: 0
Episode: 100
Episode: 200
Episode: 300
Episode: 400
Episode: 500
Episode: 600
Episode: 700
Episode: 800
Episode: 900
Episode: 1000
Episode: 1100
Episode: 1200
Episode: 1300
Episode: 1400
Episode: 1500
Episode: 1600
Episode: 1700
Episode: 1800
Episode: 1900
Episode: 2000
Episode: 2100
Episode: 2200
Episode: 2300
Episode: 2400
Episode: 2500
Episode: 2600
Episode: 2700
Episode: 2800
Episode: 2900
Episode: 3000
Episode: 3100
Episode: 3200
Episode: 3300
Episode: 3400
Episode: 3500
Episode: 3600
Episode: 3700
Episode: 3800
Episode: 3900
Episode: 4000
Episode: 4100
Episode: 4200
Episode: 4300
Episode: 4400
Episode: 4500
Episode: 4600
Episode: 4700
Episode: 4800
Episode: 4900
Episode: 5000
Episode: 5100
Episode: 5200
Episode: 5300
Episode: 5400
Episode: 5500
Episode: 5600
Episode: 5700
Episode: 5800
Episode: 5900
Episode: 6000
Episode: 6100
Episode: 6200
Episode: 6300
Episode: 6400
Episode: 6500
Episode: 6600
Episode: 6700
Episode: 6800
Episode: 6900
Episode: 7000
Episode: 7100
Epis

In [1]:
from game2 import EKGameV0, Cards
from collections import defaultdict
import random
import pickle
from datetime import date
import pandas as pd

def def_value():
    return defaultdict(0.0)

class Experience:
    def __init__(self, s, a, r) -> None:
        self.s = s
        self.a = a
        self.r = r

class SARSALambda:
    def __init__(self, g, Q=None, traces=None) -> None:
        self.g = g
        self.action_space = range(g.action_size)
        self.discount_factor = 0.99
        self.Q = defaultdict(lambda: defaultdict(int)) if Q is None else Q
        self.traces = defaultdict(lambda: defaultdict(int)) if traces is None else traces
        self.visits =  defaultdict(lambda: defaultdict(int))
        self.learning_rate = 0.01
        self.trace_decay_rate = 0.75
        self.last_experience = None
        self.espilon = 0.9

    def update_sarsa_lambda(self, s, a, r):
        if self.last_experience != None:
            s1 = self.last_experience.s
            a1 = self.last_experience.a
            r1 = self.last_experience.r
            self.traces[s1][a1] += 1
            self.visits[s1][a1] += 1
            td_update = r1 + self.discount_factor * self.Q[s][a] - self.Q[s1][a1]
            for _s in self.Q:
                for _a in self.Q[_s]:
                    self.Q[_s][_a] += self.learning_rate * td_update * self.traces[_s][_a]
                    self.traces[_s][_a] *= self.discount_factor * self.trace_decay_rate
        self.last_experience = Experience(s, a, r)

    def update_sarsa(self, s, a, r):
        if self.last_experience != None:
            s1 = self.last_experience.s
            a1 = self.last_experience.a
            r1 = self.last_experience.r
            self.visits[s1][a1] += 1
            self.Q[s1][a1] += self.learning_rate * (r1 + self.discount_factor * self.Q[s][a] - self.Q[s1][a1])
        self.last_experience = Experience(s, a, r)

    def update_qlearning(self, s, a, r, sp):
        # if self.last_experience != None:
        #     s1 = self.last_experience.s
        #     a1 = self.last_experience.a
        #     r1 = self.last_experience.r
        #     self.visits[s1][a1] += 1
        #     q_vals = self.Q[s].values()
        #     # print(q_vals)
        #     self.Q[s1][a1] += self.learning_rate * (r1 + self.discount_factor * (max(q_vals) if q_vals else 0) - self.Q[s1][a1])
        #     if r1 != 0:
        #         print("NONZERO:", self.Q[s1])
        # self.last_experience = Experience(s, a, r)

        self.visits[s][a] += 1
        q_vals = self.Q[sp].values()
        self.Q[s][a] += self.learning_rate * (r + self.discount_factor * (max(q_vals) if q_vals else 0) - self.Q[s][a])

        
        
    def greedy_action(self, s):
        if len(self.Q[s]) == 0:
            return self.g.getRandomAction()
            
        allowedActions = self.g.allowedActions
        actions = [(v, a) for a, v in self.Q[s].items()]
        actions.sort(reverse=True)
        for (v, a) in actions:
            if a in allowedActions:
                return a
        else:
            return self.g.getRandomAction()

    def custom_action(self, s):
        # print("=======", 0.9^sum(self.visits[s].values()))
        if len(self.Q[s]) == 0:
            # Obs never seen before, get completely random action
            return self.g.getRandomAction()
        elif len(self.Q[s]) < len(self.g.allowedActions):
            # Pick a random action from the remaining unexplored actions until no unexplored actions are left
            visited_actions = self.Q[s].keys()
            action_candidates = [a for a in self.g.allowedActions if a not in visited_actions]
            return random.choice(action_candidates)
        elif random.uniform(0, 1) < pow(0.9, sum(self.visits[s].values())):
            # With decaying likelihood, select a random action
            # TODO try UCB
            return self.g.getRandomAction()
        else:
            # Converge towards selecting the highest value action
            return self.greedy_action(s)
            
    def epsilon_greedy_action(self, s):
        self.espilon *= 0.999999
        if len(self.Q[s]) == 0 or random.uniform(0, 1) < self.espilon:
            return self.g.getRandomAction()
        else:
            return self.greedy_action(s)

    def save(self, savefile=None):
        save_obj = {
            "Q": dict(self.Q),
            "traces": dict(self.traces),
            "visits": dict(self.visits),
            "action_space": range(self.g.action_size),
            "discount_factor": self.discount_factor,
            "learning_rate": self.learning_rate,
            "trace_decay_rate": self.trace_decay_rate,
            "espilon": self.espilon
        }

        if savefile == None:
            today = date.today()
            d = today.strftime("%b-%d-%Y")
            savefile = 'sarsaLambda-{}'.format(d)
        with open(savefile, 'wb') as f:
            pickle.dump(save_obj, f)
            # pickle.dump(self.espilon, f)

    def load(self, loadFile) -> None:
        with open(loadFile, 'rb') as f:
            save_obj = pickle.load(f)
            self.Q = defaultdict(lambda: defaultdict(int), save_obj["Q"])
            self.traces = defaultdict(lambda: defaultdict(int), save_obj["traces"])
            self.visits = defaultdict(lambda: defaultdict(int), save_obj["visits"]),
            self.action_space = save_obj["visits"]
            self.discount_factor = save_obj["discount_factor"]
            self.learning_rate = save_obj["learning_rate"]
            self.trace_decay_rate = save_obj["trace_decay_rate"]
            self.espilon = save_obj["espilon"]

def pretty_print_obs(obs_str):
    c = obs_str.split('|')
    h1 = c[0]
    n_opp = c[1]
    n_deck = c[2]
    n_discard = c[3]
    card_order = ['DEFUSE',
                'ATTACK',
                'SKIP',
                'FAVOR',
                'SHUFFLE',
                'CAT1',
                'CAT2',
                'CAT3',
                'CAT4',
                'CAT5']
    
    table = [h1.split(',')]
    df = pd.DataFrame(table, columns=card_order, index=['player 0'])
    print(df)
    print('\tNum opponent cards: ' + n_opp)
    print('\tNum deck cards: ' + n_deck)
    print('\tDiscard pile: ' + n_discard)

def pretty_print_action_values(action_values_dict):
    actions = []
    values = []
    for (k, v) in action_values_dict.items():
        actions.append(Cards(k))
        values.append(v)
    table = [values]
    df = pd.DataFrame(table, columns=actions, index=['values'])
    print(df)

def print_highest_q_val(Q):
    max_q = None
    max_s = None
    max_a = None
    for s in Q:
        for a in Q[s]:
            if not max_q or Q[s][a] > max_q:
                max_q = Q[s][a]
                max_s = s
                max_a = a
    if max_q:
        print("Max Q val:")
        pretty_print_obs(max_s)
        pretty_print_action_values(Q[max_s])

game = EKGameV0()
model = SARSALambda(game)
num_ep = 10000
num_games_total = 0
games_won_total = 0
num_games_this_period = 0
games_won_this_period = 0
debug = False
print_percent = 5
for i in range(num_ep):
    if i % (num_ep / (100 / print_percent) ) == 0:
        print('Episode: ', i)
        print("\tespilon", model.espilon)
        print("\tstates", len(model.Q))
        print("\tnum_games_total", num_games_total)
        print("\tgames_won_total", games_won_total)
        print("\tnum_games_this_period", num_games_this_period)
        print("\tgames_won_this_period", games_won_this_period)
        print_highest_q_val(model.Q)
        num_games_this_period = 0
        games_won_this_period = 0
        qs = list(model.Q.items())
        if qs:
            max_action_item = max(qs, key=lambda x: len(x[1]))
            print("obs with most explored action space:")
            pretty_print_obs(max_action_item[0])
            print("Action values for above obs:")
            pretty_print_action_values(max_action_item[1])

            qs = list(model.Q.items())
            print(len(qs), qs[0], len(qs[0][1]))
            print("RANDOMs")
            for _ in range(10):
                action, value = random.choice(qs)
                print(action, value)

            action, value = max(qs, key=lambda x: len(x[1]))
            print("MAX")
            print(action, value)
            print(model.espilon)

    state = game.reset()
    model.last_experience = None
    done = False
    while not done:
        # action = model.epsilon_greedy_action(state)
        action = model.custom_action(state)
        (new_state, reward, done, info) = game.step(action)
        if model.espilon > 0.8:
            model.update_qlearning(state, action, reward, new_state)
        else:
            model.update_qlearning(state, action, reward, new_state)
        state = new_state
    # print("END REWARD:", reward)

    num_games_total += 1
    num_games_this_period += 1
    if reward == 1:
        games_won_total += 1
        games_won_this_period += 1

today = date.today()
d = today.strftime("%b-%d-%Y")
modelFile = 'models/sarsaLambda-{}'.format(d)

model.save(modelFile)

newGame = EKGameV0()
saved_model = SARSALambda(newGame)
saved_model.load(modelFile)



Episode:  0
	espilon 0.9
	states 0
	num_games_total 0
	games_won_total 0
	num_games_this_period 0
	games_won_this_period 0
Episode:  500
	espilon 0.9
	states 4706
	num_games_total 500
	games_won_total 257
	num_games_this_period 500
	games_won_this_period 257
Max Q val:
         DEFUSE ATTACK SKIP FAVOR SHUFFLE CAT1 CAT2 CAT3 CAT4 CAT5
player 0      2      0    0     0       0    1    1    0    0    0
	Num opponent cards: 0
	Num deck cards: 1
	Discard pile: 0,2,2,2,2,2,2,0,0,0
        Cards.NULL
values    0.077255
obs with most explored action space:
         DEFUSE ATTACK SKIP FAVOR SHUFFLE CAT1 CAT2 CAT3 CAT4 CAT5
player 0      1      1    1     1       1    0    0    0    0    0
	Num opponent cards: 5
	Num deck cards: 9
	Discard pile: 0,0,0,0,0,0,0,0,0,0
Action values for above obs:
        Cards.ATTACK  Cards.SHUFFLE  Cards.FAVOR  Cards.NULL
values           0.0            0.0          0.0         0.0
4706 ('2,0,1,0,1,0,1,0,0,0|5|9|0,0,0,0,0,0,0,0,0,0', defaultdict(<class 'int'>, {1

In [1]:
game = newGame
model = saved_model
model.espilon = 0.0024965314697670215
num_ep = 5000000
num_games_total = 0
games_won_total = 0
num_games_this_period = 0
games_won_this_period = 0
debug = False
print_percent = 1
for i in range(num_ep):
    if i % (num_ep / (100 / print_percent) ) == 0:
        print('Episode: ', i)
        print("\tespilon", model.espilon)
        print("\tstates", len(model.Q))
        print("\tnum_games_total", num_games_total)
        print("\tgames_won_total", games_won_total)
        print("\tnum_games_this_period", num_games_this_period)
        print("\tgames_won_this_period", games_won_this_period)
        num_games_this_period = 0
        games_won_this_period = 0
        qs = list(model.Q.items())
        if qs:
            max_action_item = max(qs, key=lambda x: len(x[1]))
            print("obs with most explored action space:")
            pretty_print_obs(max_action_item[0])
            print("Action values for above obs:")
            pretty_print_action_values(max_action_item[1])

    state = game.reset()
    done = False
    while not done:
        action = model.epsilon_greedy_action(state)
        (new_state, reward, done, info) = game.step(action)
        if model.espilon > 0.8:
            model.update_sarsa_lambda(state, action, reward)
        else:
            model.update_sarsa(state, action, reward)
        state = new_state

    num_games_total += 1
    num_games_this_period += 1
    if reward == 1:
        games_won_total += 1
        games_won_this_period += 1

today = date.today()
d = today.strftime("%b-%d-%Y")
modelFile = 'models/sarsaLambda-v6-{}'.format(d)

model.save(modelFile)

newGame = EKGameV0()
saved_model = SARSALambda(newGame)
saved_model.load(modelFile)

NameError: name 'newGame' is not defined

In [9]:
qs = list(model.Q.items())
print(len(qs), qs[0], len(qs[0][1]))
# qs.sort(key=lambda x: max(x[1].values()))
# print("MINs")
# for i in range(10): 
#     print(qs[i])
# print("MAXs")
# for i in range(1, 11): 
#     print(qs[-i])
print("RANDOMs")
for _ in range(10):
    action, value = random.choice(qs)
    print(action, value)

action, value = max(qs, key=lambda x: len(x[1]))
print("MAX")
print(action, value)
print(model.espilon)

12691 ('1,0,0,1,0,1,1,0,0,0|4|4|0,0,0,0,0,0,0,0,0,0', defaultdict(<class 'int'>, {3: 0.0, 10: 0.0})) 2
RANDOMs
2,0,1,0,0,0,0,0,0,0|0|2|0,1,0,1,0,2,2,0,0,0 defaultdict(<class 'int'>, {2: 0.00039278467732158117, 10: 0.11988907326023841})
1,1,0,1,0,2,1,0,0,0|2|2|0,0,1,0,0,0,0,0,0,0 defaultdict(<class 'int'>, {3: 0.11310108851905297, 10: 1.883190425980308, 5: 0.13181214120761098, 1: 0.15652713791006168})
2,0,0,0,0,1,0,0,0,0|3|4|0,0,0,0,0,0,2,0,0,0 defaultdict(<class 'int'>, {10: 0.25721579500288866})
1,0,0,0,0,0,0,0,0,0|4|2|0,1,1,1,0,0,2,0,0,0 defaultdict(<class 'int'>, {10: 0.0})
0,0,0,0,1,2,2,0,0,0|2|1|0,1,1,1,0,0,0,0,0,0 defaultdict(<class 'int'>, {4: 0.0, 6: 0.0, 10: -0.0490099501, 5: 0.0})
1,0,0,0,0,0,0,0,0,0|5|2|0,1,1,0,0,2,0,0,0,0 defaultdict(<class 'int'>, {10: 0.0})
1,0,1,0,1,0,0,0,0,0|2|1|0,1,0,1,0,2,2,0,0,0 defaultdict(<class 'int'>, {4: 0.0, 10: 0.0, 2: 0.0})
1,0,0,0,0,0,0,0,0,0|4|2|0,1,0,1,1,0,2,0,0,0 defaultdict(<class 'int'>, {10: 0.0})
2,0,0,0,0,0,0,0,0,0|5|3|0,1,0,0,1,0,0,

In [2]:
from game2 import EKGameV0

num_random = 0
num_turns = 0

def greedy_action(model, s, num_random, num_turns):
    num_turns += 1

    all_zeros = True
    for a in model.Q[s]:
        if model.Q[s][a] != 0:
            all_zeros = False
            break
    if all_zeros or len(model.Q[s]) == 0:
        num_random += 1

    if len(model.Q[s]) == 0:
        return (model.g.getRandomAction(), num_random, num_turns)
        
    allowedActions = model.g.allowedActions
    actions = [(v, a) for a, v in model.Q[s].items()]
    actions.sort(reverse=True)
    for (v, a) in actions:
        if a in allowedActions:
            return (a, num_random, num_turns)
    else:
        num_random += 1
        return (model.g.getRandomAction(), num_random, num_turns)

g = EKGameV0()
saved_model.g = g
num_won = 0
num_bad = 0
num_ep = 10000
for i in range(num_ep):
    if i % 100 == 0:
        print(i)
        print("\tnum_won", num_won)
        print("\tnum_bad", num_bad)
        print("\tnum_random", num_random)
        print("\tnum_turns", num_turns)
    state = g.reset()
    done = False
    while not done:
        (action, num_random, num_turns) = greedy_action(saved_model, state, num_random, num_turns)
        (new_state, reward, done, info) = g.step(action)
        state = new_state

    if reward == 1:
        num_won += 1
    elif reward == 0:
        num_bad += 1

print("num_won", num_won)
print("num_bad", num_bad)
print("num_random", num_random)
print("num_turns", num_turns)

0
	num_won 0
	num_bad 0
	num_random 0
	num_turns 0
100
	num_won 59
	num_bad 0
	num_random 30
	num_turns 1015
200
	num_won 123
	num_bad 0
	num_random 62
	num_turns 2014
300
	num_won 192
	num_bad 0
	num_random 110
	num_turns 3006
400
	num_won 250
	num_bad 0
	num_random 150
	num_turns 4026
500
	num_won 319
	num_bad 0
	num_random 184
	num_turns 5010
600
	num_won 380
	num_bad 0
	num_random 234
	num_turns 6005
700
	num_won 435
	num_bad 0
	num_random 270
	num_turns 6988
800
	num_won 501
	num_bad 0
	num_random 305
	num_turns 8016
900
	num_won 560
	num_bad 0
	num_random 319
	num_turns 9020
1000
	num_won 612
	num_bad 0
	num_random 352
	num_turns 9999
1100
	num_won 675
	num_bad 0
	num_random 409
	num_turns 11026
1200
	num_won 740
	num_bad 0
	num_random 445
	num_turns 12055
1300
	num_won 797
	num_bad 0
	num_random 500
	num_turns 13062
1400
	num_won 861
	num_bad 0
	num_random 542
	num_turns 14090
1500
	num_won 921
	num_bad 0
	num_random 560
	num_turns 15136
1600
	num_won 979
	num_bad 0
	num_random 

In [ ]:
num_won 546
num_bad 0
num_random 5029
num_turns 10008